In [1]:
import numpy as np
import dask as d
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import keras_tuner as kt
import multiprocessing
import os

from glob import glob
from copy import deepcopy
from tensorflow import data
from tensorflow.keras import Sequential, Input, Model
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, Softmax
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import BinaryAccuracy
from tensorflow.keras.utils import Sequence
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications import resnet50

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
from fl_tissue_model_tools import data_prep, dev_config, models, defs
import fl_tissue_model_tools.preprocessing as prep

In [3]:
n_cores = multiprocessing.cpu_count()
n_cores

16

In [4]:
dirs = dev_config.get_dev_directories("../dev_paths.txt")

In [5]:
root_data_path = f"{dirs.data_dir}/invasion_data/development"
seed = 2049
resnet_inp_shape = (128, 128, 3)
# Binary classification -> only need 1 output unit
n_outputs = 1
val_split = 0.2
batch_size = 32
frozen_epochs = 20
fine_tune_epochs = 20
adam_beta_1_range = (0.85, 0.95)
adam_beta_2_range = (0.98, 0.999)
frozen_lr_range = (1e-4, 1e-2)
fine_tune_lr_range = (1e-5, 1e-3)
last_layer_options = [
    "conv5_block3_out",
    "conv5_block2_out",
    "conv5_block1_out",
    "conv4_block6_out",
    "conv4_block5_out",
    "conv4_block4_out",
    "conv4_block3_out",
    "conv4_block2_out",
    "conv4_block1_out"
]
# 5 hyperparamters, 3 * 5 is opt default, 3 times as many as default
max_opt_trials = 3 * (3 * 5)
class_labels = {"no_invasion": 0, "invasion": 1}
project_name = "invasion_hp_trials"
hypermodel_name = "invasion_depth_hypermodel"

# Early stopping
es_criterion = "val_loss"
es_mode = "min"
# Update these depending on seriousness of experiment
es_patience = 5
es_min_delta = 0.0001

# Frozen model saving (for transitioning from frozen model to fine-tuned model)
mcp_criterion = "val_loss"
mcp_mode = "min"
mcp_best_frozen_weights_path = "../model_training/resnet50_invasion_depth_hp_demo_v1_best_frozen_weights.h5"

# Prep for loading data

In [6]:
rs = np.random.RandomState(seed)

In [7]:
data_paths = {v: glob(f"{root_data_path}/train/{k}/*.tif") for k, v in class_labels.items()}
for k, v in data_paths.items():
    rs.shuffle(v)

In [8]:
data_counts = {k: len(v) for k, v in data_paths.items()}
val_counts = {k: round(v * val_split) for k, v in data_counts.items()}
train_counts = {k: v - val_counts[k] for k, v in data_counts.items()}
train_class_weights = prep.balanced_class_weights_from_counts(train_counts)

In [9]:
data_counts

{0: 403, 1: 47}

In [10]:
train_class_weights

{0: 0.5590062111801242, 1: 4.7368421052631575}

In [11]:
train_data_paths = {k: v[val_counts[k]:] for k, v in data_paths.items()}
val_data_paths = {k: v[:val_counts[k]] for k, v in data_paths.items()}

# Datasets

In [12]:
class InvasionDataGenerator(Sequence):
    def __init__(self, data_paths, class_labels, batch_size, img_shape, random_state, class_weights=None, shuffle=True, augmentation_function=None):
        self.data_paths = deepcopy(data_paths)
        self.batch_size = batch_size
        self.img_shape = img_shape
        self.class_labels = deepcopy(class_labels)
        self.class_paths = {}
        self.class_counts = {}
        self.img_paths = []
        self.img_labels = []
        self.shuffle = shuffle
        self.rs = random_state
        self.augmentation_function = augmentation_function
        self._get_paths_and_counts(data_paths)
        self.indices = np.arange(len(self.img_paths), dtype=np.uint)
        if class_weights != None:
            self.class_weights = deepcopy(class_weights)
        else:
            self.class_weights = None
        self.shuffle_indices()

    def __len__(self):
        # return len()
        return len(self.img_paths) // self.batch_size

    def __getitem__(self, index):        
        batch_idx_start = index * self.batch_size
        batch_idx_end = batch_idx_start + batch_size
        batch_indices = self.indices[batch_idx_start: batch_idx_end]

        img_paths = [self.img_paths[i] for i in batch_indices]
        # Should it be (B,) or (B,1)?
        y = np.array([self.img_labels[i] for i in batch_indices])

        # Generate data
        X = self.prep_images(img_paths)
        
        if self.augmentation_function != None:
            X = self.augmentation_function(X, self.rs, expand_dims=False)
        
        if self.class_weights != None:
            # Weight classes by relative proportions in the training set
            w = np.array([self.class_weights[y_] for y_ in y])
            return X, y, w

        return X, y

    
    def _get_paths_and_counts(self, data_paths):
        self.class_paths = deepcopy(data_paths)
        self.class_counts = {c: len(pn) for c, pn in self.class_paths.items()}
        for k, v in self.class_paths.items():
            # Paths to each image
            self.img_paths.extend(v)
            # Associate labels with each image path
            self.img_labels.extend(list(np.repeat(k, len(v))))
            
    def _load_img(self, path):
        img = cv2.imread(path, cv2.IMREAD_ANYDEPTH)
        img = prep.min_max_(cv2.resize(img, self.img_shape, cv2.INTER_LANCZOS4).astype(np.float32), defs.GS_MIN, defs.GS_MAX, defs.TIF_MIN, defs.TIF_MAX)
        img = np.repeat(img[:, :, np.newaxis], 3, axis=2)
        return img
            
    def shuffle_indices(self):
        # print("shuffling")
        self.rs.shuffle(self.indices)
    
    def on_epoch_end(self):
        self.indices = np.arange(len(self.img_paths), dtype=np.uint)
        if self.shuffle == True:
            self.shuffle_indices()

    def prep_images(self, paths):
        imgs = np.array(d.compute((d.delayed(self._load_img)(p) for p in paths))[0])
        return resnet50.preprocess_input(imgs)

In [13]:
# With class weights
train_datagen = InvasionDataGenerator(train_data_paths, class_labels, batch_size, resnet_inp_shape[:2], rs, class_weights=train_class_weights, augmentation_function=prep.augment_imgs)
val_datagen = InvasionDataGenerator(val_data_paths, class_labels, batch_size, resnet_inp_shape[:2], rs, class_weights=train_class_weights, augmentation_function=prep.augment_imgs)
# # Without class weights
# train_datagen = InvasionDataGenerator(train_data_paths, class_labels, batch_size, resnet_inp_shape[:2], rs, augmentation_function=prep.augment_imgs)
# val_datagen = InvasionDataGenerator(val_data_paths, class_labels, batch_size, resnet_inp_shape[:2], rs, augmentation_function=prep.augment_imgs)

# Build hyper model

In [14]:
hypermodel = models.ResNet50TLHyperModel(
    n_outputs=n_outputs,
    img_shape=resnet_inp_shape,
    loss=BinaryCrossentropy(),
    metrics=[BinaryAccuracy()],
    name=hypermodel_name,
    output_act="sigmoid",
    adam_beta_1_range=adam_beta_1_range,
    adam_beta_2_range=adam_beta_2_range,
    frozen_lr_range=frozen_lr_range,
    fine_tune_lr_range=fine_tune_lr_range,
    frozen_epochs=frozen_epochs,
    fine_tune_epochs=fine_tune_epochs,
    base_model_name="base_model",
    # EarlyStopping callback parameters
    es_criterion=es_criterion,
    es_mode=es_mode,
    es_patience=es_patience,
    es_min_delta=es_min_delta,
    # Frozen ModelCheckpoint callback parameters
    mcp_criterion=mcp_criterion,
    mcp_mode=mcp_mode,
    mcp_best_frozen_weights_path=mcp_best_frozen_weights_path
)

In [15]:
tuner = kt.BayesianOptimization(
    hypermodel=hypermodel,
    objective="val_loss",
    max_trials=max_opt_trials,
    seed=seed,
    directory="../model_training/",
    project_name=project_name
)

In [16]:
# Cannot use external callbacks. Callbacks are defined inside the hypermodel's fit function
tuner.search(
    train_datagen,
    validation_data=val_datagen,
    workers=n_cores
)

Trial 32 Complete [00h 02m 20s]
val_loss: 0.27601462602615356

Best val_loss So Far: 0.20104452967643738
Total elapsed time: 01h 15m 06s

Search: Running Trial #33

Hyperparameter    |Value             |Best Value So Far 
last_resnet_layer |conv5_block2_out  |conv5_block3_out  
frozen_lr         |0.0001            |0.0001            
adam_beta_1       |0.8677            |0.88278           
adam_beta_2       |0.999             |0.999             
fine_tune_lr      |1e-05             |1e-05             

Epoch 1/20
11/11 [==============================] - 9s 363ms/step - loss: 0.9329 - binary_accuracy: 0.8558 - val_loss: 0.7283 - val_binary_accuracy: 0.8906
Epoch 2/20
11/11 [==============================] - 5s 272ms/step - loss: 0.8045 - binary_accuracy: 0.7784 - val_loss: 0.8309 - val_binary_accuracy: 0.8281
Epoch 3/20
11/11 [==============================] - 4s 256ms/step - loss: 0.7326 - binary_accuracy: 0.6790 - val_loss: 0.6657 - val_binary_accuracy: 0.6875
Epoch 4/20
11/11 [======

KeyboardInterrupt: 

In [ ]:
tuner.results_summary()

In [ ]:
best_hp = tuner.get_best_hyperparameters()[0]

In [ ]:
best_hp.values

In [ ]:
best_tl_model = tuner.get_best_models()[0]

In [ ]:
best_tl_model.summary()